In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from pathlib import Path

In [4]:
abc = !pwd
isLocal = (abc[0].find('anooshhari') != 0)

In [5]:
if isLocal:
    base_directory = "/Users/anooshhari/Documents/Anoosh/Masters/Spring 21/Classes/6100 - BigData Analytics for C/Project/indoor-location-navigation"
    base_git = base_directory
    test_directory = "/Users/anooshhari/Documents/Anoosh/Masters/Spring 21/Classes/6100 - BigData Analytics for C/Project/indoor-location-navigation/test"
else:
    base_directory = "/kaggle/input/indoor-location-navigation/"
    base_git = "/kaggle/working/"

In [6]:
if not isLocal:
    !git clone https://github.com/location-competition/indoor-location-competition-20.git

In [7]:
import sys
sys.path.append(base_git +'/indoor-location-competition-20')
from io_f import read_data_file

In [8]:
import glob

txtfiles = []
for file in glob.glob(f"{base_directory}/train/*/*/*.txt"):
    lst = file.split("/")[-3:]
    lst.append(file)
    txtfiles.append(lst)

In [9]:
df = pd.DataFrame(txtfiles,columns=['site_id','floor','path_id',"file_path"])
df["file_data"] = None

In [10]:
site1_b1 = df[df.site_id.eq('5a0546857ecc773753327266')]

In [11]:
site1_b1.head()

,site_id,floor,path_id,file_path,file_data
0,5a0546857ecc773753327266,F4,5d79be0d43d77a0006b63a6d.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None
1,5a0546857ecc773753327266,F4,5d79be0043d77a0006b63a5f.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None
2,5a0546857ecc773753327266,F4,5d11dc04ffe23f0008604f57.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None
3,5a0546857ecc773753327266,F4,5d8f0a4db6e29d0006fb8c36.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None
4,5a0546857ecc773753327266,F4,5d8f0a52d5bae80006eb8de3.txt,/Users/anooshhari/Documents/Anoosh/Masters/Spr...,None


In [12]:
def remove_timestamp_column(data):
    for k,v in data.items():
        if k in ['acce','gyro','magn','ahrs']:
            data[k] = v[:,1:]
    return data

In [13]:
# read_data_file(site1_b1.file_path.values[0])
for i,row in site1_b1.iterrows():
    readData = read_data_file(row.file_path)
    row.file_data = pd.DataFrame.from_dict({k:list(v) for k,v in remove_timestamp_column(readData.__dict__).items()}, orient='index', dtype=object).transpose()

In [ ]:
# pd.DataFrame.from_dict({k:list(v) for k,v in read_data_file(site1_b1.file_path.values[0]).__dict__.items()}, orient='index').transpose()

In [ ]:
site1_b1.file_data.values[0]['acce'].values[0][:,1:]

In [ ]:
from main import calibrate_magnetic_wifi_ibeacon_to_position

In [ ]:
site1_b1.file_path[0:1].values[0]

In [ ]:
single_path_data = calibrate_magnetic_wifi_ibeacon_to_position(site1_b1.file_path[0:1])

In [ ]:
len(single_path_data.keys())

In [ ]:
# test_single_path_data = calibrate_magnetic_wifi_ibeacon_to_position(["/Users/anooshhari/Documents/Anoosh/Masters/Spring 21/Classes/6100 - BigData Analytics for C/Project/indoor-location-navigation/test/046cfa46be49fc10834815c6.txt"])

# caliberate magnetic wifi method cannot be called without waypoints i.e., on test file

In [ ]:
list(site1_b1.file_data.values[0].acce.values[0][:,0])

In [ ]:
site1_b1.file_data.values[0][['acce','gyro','magn','ahrs']].shape

In [ ]:
site1_b1.file_data.values[0][['acce','gyro','magn','ahrs']].dropna().shape

In [14]:
data_combined = pd.DataFrame(columns=['acce','gyro','magn','ahrs','floor'])
print(data_combined)
for i,data in site1_b1.iterrows():
    current = data.file_data[['acce','gyro','magn','ahrs']].dropna()
    current['floor'] = data.floor
    current.reset_index(drop=True)
#     print(type(data))
#     print(type(current))
#     break
    data_combined = pd.concat([data_combined, current])
# print(data_combined)

Empty DataFrame
Columns: [acce, gyro, magn, ahrs, floor]
Index: []


In [ ]:
# data_combined.describe()

In [15]:
data_combined.head()

,acce,gyro,magn,ahrs,floor
0,"[-1.4918823, 0.34014893, 10.652069]","[0.2646942, -0.29737854, 0.27461243]","[-23.14148, -18.16864, -23.690796]","[0.06200746, 0.026087942, -0.87248856]",F4
1,"[-1.5619049, 0.4065857, 10.006226]","[0.2641449, -0.27552795, 0.22401428]","[-25.195312, -19.537354, -25.686646]","[0.06252557, 0.021754283, -0.8709258]",F4
2,"[-1.6876068, 0.5298767, 9.641113]","[0.2055664, -0.17752075, 0.21813965]","[-25.195312, -17.48352, -25.686646]","[0.067413874, 0.016335025, -0.86725426]",F4
3,"[-1.6343384, 0.6310425, 9.501663]","[0.15975952, -0.05343628, 0.24797058]","[-23.14148, -17.48352, -23.690796]","[0.067413874, 0.016335025, -0.86725426]",F4
4,"[-1.5900421, 0.7214203, 9.437012]","[0.17041016, 0.004638672, 0.25810242]","[-23.14148, -19.537354, -24.35608]","[0.07057936, 0.014525489, -0.8652526]",F4


In [16]:
list(data_combined.floor.value_counts().keys())

['F2', 'F4', 'B1', 'F1', 'F3']

In [17]:
data_combined_copy = data_combined.copy()
data_combined_copy.replace({"floor":{'F2':1, 'F4':3, 'B1':-1, 'F1':0, 'F3':2}}, inplace=True)

In [18]:
data_combined_copy.head()

,acce,gyro,magn,ahrs,floor
0,"[-1.4918823, 0.34014893, 10.652069]","[0.2646942, -0.29737854, 0.27461243]","[-23.14148, -18.16864, -23.690796]","[0.06200746, 0.026087942, -0.87248856]",3
1,"[-1.5619049, 0.4065857, 10.006226]","[0.2641449, -0.27552795, 0.22401428]","[-25.195312, -19.537354, -25.686646]","[0.06252557, 0.021754283, -0.8709258]",3
2,"[-1.6876068, 0.5298767, 9.641113]","[0.2055664, -0.17752075, 0.21813965]","[-25.195312, -17.48352, -25.686646]","[0.067413874, 0.016335025, -0.86725426]",3
3,"[-1.6343384, 0.6310425, 9.501663]","[0.15975952, -0.05343628, 0.24797058]","[-23.14148, -17.48352, -23.690796]","[0.067413874, 0.016335025, -0.86725426]",3
4,"[-1.5900421, 0.7214203, 9.437012]","[0.17041016, 0.004638672, 0.25810242]","[-23.14148, -19.537354, -24.35608]","[0.07057936, 0.014525489, -0.8652526]",3


In [98]:
# data_combined_copy = pd.get_dummies(data_combined, columns=["floor"], prefix=["floor"])

In [17]:
data_combined_copy.acce = data_combined_copy.acce.apply(lambda x: x/np.sqrt((x**2).sum()))
data_combined_copy.gyro = data_combined_copy.gyro.apply(lambda x: x/np.sqrt((x**2).sum()))
data_combined_copy.magn = data_combined_copy.magn.apply(lambda x: x/np.sqrt((x**2).sum()))
data_combined_copy.ahrs = data_combined_copy.ahrs.apply(lambda x: x/np.sqrt((x**2).sum()))
# data_combined_copy[['acce','gyro','magn','ahrs']] = data_combined_copy[['acce','gyro','magn','ahrs']].apply(lambda x: x/np.sqrt((x**2).sum()))

In [18]:
data_combined_copy.shape

(1019140, 5)

In [88]:
# !pip install keras
# !pip install tensorflow
# !pip install layers

  Created wheel for layers: filename=layers-0.1.5-py3-none-any.whl size=5378 sha256=f060d205eb6c5116ae73638eb33ea4253cf7aaebc8cfb4f28878b4ea7b8f6e9d
  Stored in directory: /Users/anooshhari/Library/Caches/pip/wheels/1e/ab/25/39853b8da8d6de047d8f82044a317d8f1042adedd4fbca7a1b
  Created wheel for bashutils: filename=Bashutils-0.0.4-py3-none-any.whl size=5472 sha256=da5304b96aaf93bf3b87e179eca1d007e2277e6eafc8e95731a926c97c87bd50
  Stored in directory: /Users/anooshhari/Library/Caches/pip/wheels/d3/8c/b6/84627055e4d7e483978e767c370b1c501d3bc4c60bac616bbf
Successfully built layers bashutils


In [19]:
import keras
from keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [20]:
train, test = train_test_split(data_combined_copy, test_size=0.4)

In [21]:
print(train.shape)
print(test.shape)

(611484, 5)
(407656, 5)


In [109]:
np.random.uniform(0,1, 3)

array([0.75531089, 0.82786262, 0.40684179])

In [79]:
# df = pd.DataFrame()
# df['tfidf'] = [[np.random.uniform(0,1, 3)],
#               [np.random.uniform(0,1, 3)],
#               [np.random.uniform(0,1, 3)],
#               [np.random.uniform(0,1, 3)]]
# df['is_similar'] = np.random.randint(0,2, 4)

In [22]:
dataset = tf.data.Dataset.from_tensor_slices((train[['acce','gyro','magn','ahrs']].values, train.floor.values))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [23]:
data_combined_junk = data_combined_copy.copy()

In [24]:
data_combined_junk[['acce_x','acce_y','acce_z']] = pd.DataFrame(data_combined_junk.acce.tolist(), index= data_combined_junk.index)

In [25]:
data_combined_junk[['gyro_x','gyro_y','gyro_z']] = pd.DataFrame(data_combined_junk.gyro.tolist(), index= data_combined_junk.index)

In [26]:
data_combined_junk[['magn_x','magn_y','magn_z']] = pd.DataFrame(data_combined_junk.magn.tolist(), index= data_combined_junk.index)

In [27]:
data_combined_junk[['ahrs_x','ahrs_y','ahrs_z']] = pd.DataFrame(data_combined_junk.ahrs.tolist(), index= data_combined_junk.index)

In [28]:
data_combined_junk.head()

,acce,gyro,magn,ahrs,floor,acce_x,acce_y,acce_z,gyro_x,gyro_y,gyro_z,magn_x,magn_y,magn_z,ahrs_x,ahrs_y,ahrs_z
0,"[-0.13863257916705385, 0.031608206268559966, 0...","[0.5472943401009259, -0.6148740388322708, 0.56...","[-0.6126284955170183, -0.48098162212573775, -0...","[0.07085933868485227, 0.029812127730579226, -0...",3,-0.138633,0.031608,0.989839,0.547294,-0.614874,0.567802,-0.612628,-0.480982,-0.627171,0.070859,0.029812,-0.997041
1,"[-0.15410160938133993, 0.040114805146868195, 0...","[0.5968395970749634, -0.6225597793517522, 0.50...","[-0.6153778079284228, -0.47718615579126794, -0...","[0.07158556108818988, 0.024906491130369073, -0...",3,-0.154102,0.040115,0.987240,0.596840,-0.622560,0.506164,-0.615378,-0.477186,-0.627378,0.071586,0.024906,-0.997123
2,"[-0.17216906371743204, 0.05405783819114892, 0....","[0.5900942479658905, -0.5095870408276394, 0.62...","[-0.629827495485245, -0.43704962311505363, -0....","[0.0774850916657818, 0.018775377149929662, -0....",3,-0.172169,0.054058,0.983583,0.590094,-0.509587,0.626187,-0.629827,-0.437050,-0.642110,0.077485,0.018775,-0.996817
3,"[-0.1691541998296951, 0.06531296648602905, 0.9...","[0.5329228385260293, -0.17825174999193594, 0.8...","[-0.6179404199141778, -0.46685750826558736, -0...","[0.0774850916657818, 0.018775377149929662, -0....",3,-0.169154,0.065313,0.983423,0.532923,-0.178252,0.827176,-0.617940,-0.466858,-0.632609,0.077485,0.018775,-0.996817
4,"[-0.16567804195428087, 0.07517002394469298, 0....","[0.5509211456888481, 0.014996420945293288, 0.8...","[-0.5954389565421598, -0.5027034433128215, -0....","[0.08128940613484963, 0.01672965544924594, -0....",3,-0.165678,0.075170,0.983311,0.550921,0.014996,0.834423,-0.595439,-0.502703,-0.626691,0.081289,0.016730,-0.996550


In [29]:
data_combined_split = data_combined_junk[data_combined_junk.columns[4:]].copy()

In [30]:
data_combined_split.head()

,floor,acce_x,acce_y,acce_z,gyro_x,gyro_y,gyro_z,magn_x,magn_y,magn_z,ahrs_x,ahrs_y,ahrs_z
0,3,-0.138633,0.031608,0.989839,0.547294,-0.614874,0.567802,-0.612628,-0.480982,-0.627171,0.070859,0.029812,-0.997041
1,3,-0.154102,0.040115,0.987240,0.596840,-0.622560,0.506164,-0.615378,-0.477186,-0.627378,0.071586,0.024906,-0.997123
2,3,-0.172169,0.054058,0.983583,0.590094,-0.509587,0.626187,-0.629827,-0.437050,-0.642110,0.077485,0.018775,-0.996817
3,3,-0.169154,0.065313,0.983423,0.532923,-0.178252,0.827176,-0.617940,-0.466858,-0.632609,0.077485,0.018775,-0.996817
4,3,-0.165678,0.075170,0.983311,0.550921,0.014996,0.834423,-0.595439,-0.502703,-0.626691,0.081289,0.016730,-0.996550


In [31]:
train, test = train_test_split(data_combined_split, test_size=0.4)

In [ ]:
def getmodel():
    
    x_input = keras.Input(shape=(None, train.shape[0], train.shape[1]-1),name="input")  # Variable-length sequence of ints
    conv1d_1= layers.Conv1D(64, 1, activation='relu')(x_input)
    flat = layers.Flatten()(conv1d_1)
    dense1 = layers.Dense(50, name="dense1", activation="relu")(flat)
    encoder_conv_dense2 = layers.Dense(50, name="40_dense", activation="relu")(dense1)
    encoder_conv_dense3 = layers.Dense(1, name="floor", activation="softmax")(encoder_conv_dense2)

    model = keras.Model(
        inputs=x_input,
        outputs=encoder_conv_dense3,
    )
    
    return model

In [32]:
train[train.columns[1:]]

,acce_x,acce_y,acce_z,gyro_x,gyro_y,gyro_z,magn_x,magn_y,magn_z,ahrs_x,ahrs_y,ahrs_z
516,-0.180021,0.226788,0.957162,0.133463,0.611130,0.780197,0.403867,-0.801018,-0.441883,0.032880,0.171471,0.984640
165,0.122372,0.471113,0.873543,0.841395,0.013728,-0.540246,0.435042,0.098664,-0.894988,0.650226,0.228477,0.724572
2609,-0.093455,0.150359,0.984204,-0.049047,0.679089,-0.732416,0.164384,0.434324,-0.885630,0.861942,0.189202,0.470382
356,-0.004088,0.119658,0.992807,0.592822,-0.220166,-0.774654,0.429617,0.281953,-0.857865,0.374053,0.038104,0.926624
1068,0.040727,0.097119,0.994439,0.397750,0.522484,-0.754192,0.736730,0.202177,-0.645254,0.029877,0.003253,0.999548
...,...,...,...,...,...,...,...,...,...,...,...,...
250,-0.078833,0.489675,0.868334,0.864302,0.324816,-0.384026,-0.603009,0.042365,-0.796609,0.545843,-0.193874,-0.815149
1577,0.001684,0.576069,0.817399,0.047900,-0.449634,-0.891928,-0.019050,0.164657,-0.986167,0.992469,-0.122215,0.008304
12460,0.001693,0.016327,0.999865,0.931768,-0.312981,-0.183985,0.452537,0.368656,-0.811975,0.300298,0.149804,0.942008
1841,0.011784,0.315741,0.948772,-0.753715,0.038902,-0.656049,0.177814,0.555099,-0.812556,0.632687,0.137972,0.762018


In [33]:
train.floor = train.floor+1
dataset = tf.data.Dataset.from_tensor_slices((train[train.columns[1:]].values, train['floor'].values))

/Users/anooshhari/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [37]:
train_dataset = dataset.shuffle(len(dataset)).batch(1)

In [35]:
def get_compiled_model():
  model_new = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
  ])

  model_new.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])
  return model_new

In [61]:
# model = getmodel()
# model.compile('adam', 'binary_crossentropy')
# history = model.fit(
#     {"input": train[train.columns[1:]]},
#     {"floor": train['floor']},
#     epochs=10,
# )

In [38]:
model_new = get_compiled_model()
model_new.fit(train_dataset, epochs=3)

Epoch 1/3
611484/611484 [==============================] - 412s 672us/step - loss: 1.2385 - accuracy: 0.4411
Epoch 2/3
611484/611484 [==============================] - 444s 724us/step - loss: 1.1961 - accuracy: 0.4665
Epoch 3/3
611484/611484 [==============================] - 452s 738us/step - loss: 1.1976 - accuracy: 0.4660 - loss: 1.1976 - accu


In [39]:
dataset_test = tf.data.Dataset.from_tensor_slices(test[test.columns[1:]].values)

In [40]:
test_predictions = model_new.predict(test[test.columns.values[1:]])

In [41]:
final_predictions = np.argmax(test_predictions, axis=1)-1

In [42]:
test.floor.values

array([-1,  2,  3, ..., -1,  1,  3])

In [43]:
final_predictions

array([-1,  2,  2, ...,  1,  1,  1])

In [44]:
values = test.floor.values == final_predictions

In [45]:
values.sum()

191999

In [46]:
values.size

407656

In [91]:
sample_csv = pd.read_csv('/Users/anooshhari/Documents/Anoosh/Masters/Spring 21/Classes/6100 - BigData Analytics for C/Project/indoor-location-navigation/sample_submission.csv')

In [92]:
sample_csv.head()

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75


In [93]:
sample_csv['site']=None
sample_csv['path']=None

In [73]:
submission_data = sample_csv.copy()
for i, row in submission_data.iterrows():
#     print(row.site_path_timestamp.split("_"))
    row.site.loc(i) = row.site_path_timestamp.split("_")[0]
#     print(row.site)
#     row['path'], row['site'] = row.site_path_timestamp.split('_')[1], row.site_path_timestamp.split('_')[0]

SyntaxError: cannot assign to function call (<ipython-input-73-dde4b696104f>, line 4)

In [94]:
submission_data = pd.DataFrame(sample_csv)

In [102]:
# for i,row in submission_data.iterrows():
#     row['site'] = row.site_path_timestamp.split("_")[0]
#     row['path'] = row.site_path_timestamp.split("_")[1]
#     print(row)
submission_data.site = submission_data.site_path_timestamp.apply(lambda x: x.split("_")[0])

In [112]:
submission_data.path = submission_data.site_path_timestamp.apply(lambda x: x.split("_")[1]+".txt")

In [111]:
submission_data.shape

(10133, 6)

In [221]:
submission_data['model_input']=None
submission_data[submission_data.site == '5a0546857ecc773753327266']

,site_path_timestamp,floor,x,y,site,path,model_input
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,None
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,None
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,None
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,None
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,None
...,...,...,...,...,...,...,...
294,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,0,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,None
295,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,0,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,None
296,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,0,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,None
297,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,0,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,None


In [272]:
def transform_file_to_dataframe(file_names):
    file_names = list(set(file_names))
    input_data = pd.DataFrame(columns=['file_name', 'file_data'])
    input_data.file_name = file_names
    for i, row in input_data.iterrows():
        readData = read_data_file(test_directory+"/"+row.file_name)
        row.file_data = pd.DataFrame.from_dict({k:list(v) \
                                                for k,v in remove_timestamp_column(readData.__dict__).items()},\
                                                orient='index', dtype=object).transpose()
#     print(input_data)
    
    data_combined = pd.DataFrame(columns=['acce','gyro','magn','ahrs'])
#     print("data combined empty",data_combined)
    for i,data in input_data.iterrows():
        current = data.file_data[['acce','gyro','magn','ahrs']].dropna()
        current.reset_index(drop=True)
        data_combined = pd.concat([data_combined, current])
        
#     print("data combined full",data_combined)
    
    data_combined.acce = data_combined.acce.apply(lambda x: x/np.sqrt((x**2).sum()))
    data_combined.gyro = data_combined.gyro.apply(lambda x: x/np.sqrt((x**2).sum()))
    data_combined.magn = data_combined.magn.apply(lambda x: x/np.sqrt((x**2).sum()))
    data_combined.ahrs = data_combined.ahrs.apply(lambda x: x/np.sqrt((x**2).sum()))
    
#     print("after normalization", data_combined)
    
    data_combined[['acce_x','acce_y','acce_z']] = pd.DataFrame\
    (data_combined.acce.tolist(), index= data_combined.index)
    
    data_combined[['gyro_x','gyro_y','gyro_z']] = pd.DataFrame\
    (data_combined.acce.tolist(), index= data_combined.index)
        
    data_combined[['magn_x','magn_y','magn_z']] = pd.DataFrame\
    (data_combined.acce.tolist(), index= data_combined.index)
    
    data_combined[['ahrs_x','ahrs_y','ahrs_z']] = pd.DataFrame\
    (data_combined.acce.tolist(), index= data_combined.index)
    
    data_combined = data_combined[['acce_x','acce_y','acce_z',\
                                   'gyro_x','gyro_y','gyro_z',\
                                   'magn_x','magn_y','magn_z',\
                                   'ahrs_x','ahrs_y','ahrs_z']]
    return data_combined

In [19]:
output_onefile = transform_file_to_dataframe(['046cfa46be49fc10834815c6.txt'])
# testing123 = pd.DataFrame(columns=['file_name', 'file_data'])

In [193]:
output_onefile

,acce_x,acce_y,acce_z,gyro_x,gyro_y,gyro_z,magn_x,magn_y,magn_z,ahrs_x,ahrs_y,ahrs_z
0,0.089237,0.480441,0.872475,0.089237,0.480441,0.872475,0.089237,0.480441,0.872475,0.089237,0.480441,0.872475
1,0.003049,0.561183,0.827686,0.003049,0.561183,0.827686,0.003049,0.561183,0.827686,0.003049,0.561183,0.827686
2,-0.046211,0.591007,0.805342,-0.046211,0.591007,0.805342,-0.046211,0.591007,0.805342,-0.046211,0.591007,0.805342
3,-0.060141,0.607536,0.792012,-0.060141,0.607536,0.792012,-0.060141,0.607536,0.792012,-0.060141,0.607536,0.792012
4,-0.008734,0.562966,0.826434,-0.008734,0.562966,0.826434,-0.008734,0.562966,0.826434,-0.008734,0.562966,0.826434
...,...,...,...,...,...,...,...,...,...,...,...,...
3326,-0.168437,0.624349,0.762769,-0.168437,0.624349,0.762769,-0.168437,0.624349,0.762769,-0.168437,0.624349,0.762769
3327,-0.212063,0.755093,0.620374,-0.212063,0.755093,0.620374,-0.212063,0.755093,0.620374,-0.212063,0.755093,0.620374
3328,-0.264557,0.664647,0.698751,-0.264557,0.664647,0.698751,-0.264557,0.664647,0.698751,-0.264557,0.664647,0.698751
3329,-0.173039,0.535233,0.826791,-0.173039,0.535233,0.826791,-0.173039,0.535233,0.826791,-0.173039,0.535233,0.826791


In [195]:
predictions = model_new.predict(output_onefile)

In [347]:
predictions

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [302]:
pd.Series(np.argmax(predictions, axis=1)-1).value_counts()[0:1].keys().values[0]

3

In [229]:
# for i, row in submission_data[submission_data.site == '5a0546857ecc773753327266'].iterrows():
#     row.model_input = transform_file_to_dataframe([row.path])

In [271]:
# for i, row in submission_data[submission_data.site == '5a0546857ecc773753327266'].iterrows():
#     abc = dict()
#     if row.path in abc.keys():
#         row.model_input = abc[row.path]
#     else:
#         row.model_input = transform_file_to_dataframe([row.path])
#         abc.add(row.path, row.model_input)
abc = dict()
def model_inputs(val):
    if val in abc.keys():
        return abc[val]
    else:
        res = transform_file_to_dataframe([val])
        abc[val] = res
        return res

In [273]:
submission_data[submission_data.site == '5a0546857ecc773753327266'].model_inputs = submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x))

<ipython-input-273-4dd63209dca5>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  submission_data[submission_data.site == '5a0546857ecc773753327266'].model_inputs = submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x))


In [264]:
# submission_data[submission_data.site == '5a0546857ecc773753327266']
# abc[1]=123
# len(list(abc.keys()))
# len(set(list(submission_data.path[submission_data.site=='5a0546857ecc773753327266'].values)))

29

In [1]:
submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x)).values

In [293]:
submission_data[submission_data.site == '5a0546857ecc773753327266'].model_input = submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x)).values

<ipython-input-293-f604da85b020>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_data[submission_data.site == '5a0546857ecc773753327266']['model_input'] = submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x)).values


In [300]:
submission_data.loc[submission_data.site.str.startswith('5a0546857ecc773753327266'), 'model_input'] = submission_data[submission_data.site == '5a0546857ecc773753327266'].path.apply(lambda x: model_inputs(x))

In [301]:
submission_data[submission_data.site == '5a0546857ecc773753327266']

,site_path_timestamp,floor,x,y,site,path,model_input
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
...,...,...,...,...,...,...,...
294,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,0,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,acce_x acce_y acce_z gyro_x ...
295,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,0,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,acce_x acce_y acce_z gyro_x ...
296,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,0,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,acce_x acce_y acce_z gyro_x ...
297,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,0,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,acce_x acce_y acce_z gyro_x ...


In [309]:
prediction_floor = dict()
def final_floor_predictions_submission(df):
    for i,row in df.iterrows():
        if row.path in prediction_floor.keys():
            row['floor'] = prediction_floor[row.path]
        else:
            predicts = model_new.predict(row.model_input)
            floor_value = pd.Series(np.argmax(predicts, axis=1)-1).value_counts()[0:1].keys().values[0]
            prediction_floor[row.path] = floor_value
            row['floor'] = floor_value
        print(row.floor)

In [2]:
final_floor_predictions_submission(submission_data[submission_data.site == '5a0546857ecc773753327266'])

In [316]:
submission_data[submission_data.site == '5a0546857ecc773753327266'].loc[:,'floor'] = 3

/Users/anooshhari/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [319]:
submission_data.loc[submission_data.site.str.startswith('5a0546857ecc773753327266'), 'floor']=3

In [321]:
submission_data[submission_data.site == '5a0546857ecc773753327266']

,site_path_timestamp,floor,x,y,site,path,model_input
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,3,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,3,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,3,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,3,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,3,75,75,5a0546857ecc773753327266,046cfa46be49fc10834815c6.txt,acce_x acce_y acce_z gyro_x ...
...,...,...,...,...,...,...,...
294,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,3,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,acce_x acce_y acce_z gyro_x ...
295,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,3,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,acce_x acce_y acce_z gyro_x ...
296,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,3,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,acce_x acce_y acce_z gyro_x ...
297,5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...,3,75,75,5a0546857ecc773753327266,ffcd9524c80c0fa5bb859eaf.txt,acce_x acce_y acce_z gyro_x ...


In [327]:
submission_data.iloc[:,0:4].to_csv('submission.csv', index=False)

In [346]:
# for data in list(abc.values()):
#     for i,row in data.iterrows():
        
model_new.predict(list(abc.values())[9])

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [350]:
np.argmax(test_predictions, axis=1)

array([0, 3, 3, ..., 2, 2, 2])